In [93]:
import pandas as pd
import urllib.request, json 

import validators  
def find_cid(drugname):
    
    drugname = drugname.strip("(OTC)")
    dic_drugname_pubmed = {}
    Found = False
    if len(drugname.split(" ")) == 1:
        url_link = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22where%22:{%22ands%22:[{%22*%22:%22" +  drugname + "%22}]},%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_text_"+drugname+"%22}"
    if len(drugname.split(" ")) == 2:
        name_part1 = drugname.split(" ")[0].strip()
        name_part2 = drugname.split(" ")[1].strip()
        url_link = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22where%22:{%22ands%22:[{%22*%22:%22" + name_part1 +"%22},{%22*%22:%22" + name_part2 + "%22}]},%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_text_" + name_part1 + "%20"+name_part2+"%22}"
    else:
        url_link = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22where%22:{%22ands%22:[{%22*%22:%22" +  drugname + "%22}]},%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_text_"+drugname+"%22}"
    
    if validators.url(url_link):    
        with urllib.request.urlopen(url_link) as url:
            data = json.load(url)
            
        return_id = ""
        dic_drugname_pubmed = {}
        for curr_drug_anno in data:  
            cid = curr_drug_anno['cid']
            cmpdname = curr_drug_anno['cmpdname']
            if type(cmpdname)!= list:
                dic_drugname_pubmed[cmpdname] = cid

                if cmpdname.upper()  == drugname.upper():
                    return_id = "PUBMED.COMPOUND:" + cid
                    Found = True
                    break
            else:
                return_id = None
        else:
            return_id = None
    
    else:
        return_id = None
        
    return(return_id)

In [29]:
data_file = open("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/Drug_Target/FDA_approved_drugs.csv")

In [30]:
drug_combination = []
single_drug = []
generic_name_single_drug = []
count_drug_combination = 0

In [31]:
for line in data_file.readlines():
    words = line.strip().split(',')
    if len(words) > 2:
        print(line)
        count_drug_combination += 1
    else:
        single_drug.append(words[0])
        generic_name_single_drug.append(words[1].strip())

ABACAVIR SULFATE, LAMIVUDINE AND ZIDOVUDINE, ABACAVIR SULFATE

ACETAMINOPHEN, ASPIRIN AND CAFFEINE, ACETAMINOPHEN (OTC)

ACETAMINOPHEN, CAFFEINE AND DIHYDROCODEINE BITARTRATE, ACETAMINOPHEN

ACETIC ACID, ACETIC ACID, GLACIAL

ACETIC ACID 0.25% IN PLASTIC CONTAINER, ACETIC ACID, GLACIAL

AMLODIPINE BESYLATE, VALSARTAN AND HYDROCHLOROTHIAZIDE, AMLODIPINE BESYLATE

BISMUTH SUBSALICYLATE, METRONIDAZOLE AND TETRACYCLINE HYDROCHLORIDE, BISMUTH SUBSALICYLATE

BROMPHENIRAMINE MALEATE, PSEUDOEPHEDRINE HYDROCHLORIDE AND DEXTROMETHORPHAN HYDROBROMIDE,

BUTALBITAL, ACETAMINOPHEN AND CAFFEINE, ACETAMINOPHEN

BUTALBITAL, ACETAMINOPHEN, CAFFEINE AND CODEINE PHOSPHATE, ACETAMINOPHEN

BUTALBITAL, ASPIRIN AND CAFFEINE, ASPIRIN

BUTALBITAL, ASPIRIN, CAFFEINE, AND CODEINE PHOSPHATE, ASPIRIN

CARBIDOPA, LEVODOPA AND ENTACAPONE, CARBIDOPA

CARISOPRODOL, ASPIRIN AND CODEINE PHOSPHATE, ASPIRIN

DETROAMP SACCHARATE, AMP ASPARTATE, DEXTROAMP SULFATE AND AMP SULFATE, AMPHETAMINE

DEXTROAMP SACCHARATE, AMP ASPART

In [32]:
len(single_drug)

3903

In [33]:
len(generic_name_single_drug)

3903

In [34]:
count_drug_combination

82

In [43]:
dic_drug_id = {}
count = 0

In [100]:
#find_cid("ODEVIXIBAT")
drugname = "POTASSIUM"
url_link = "https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=json&query={%22download%22:%22*%22,%22collection%22:%22compound%22,%22where%22:{%22ands%22:[{%22*%22:%22" +  drugname + "%22}]},%22order%22:[%22relevancescore,desc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22PubChem_compound_text_"+drugname+"%22}"
with urllib.request.urlopen(url_link) as url:
    data = json.load(url)

In [119]:
for i in set(generic_name_single_drug):
    if i != '' and (i not in dic_drug_id) and i not in ["ALCOHOL (OTC)","POTASSIUM","ALCOHOL","SODIUM ACETATE","GLYCINE" ,
                                                        "METHYLENE BLUE","ZINC CHLORIDE","STREPTOMYCIN SULFATE","RUCAPARIB CAMSYLATE",
                                                       "ZINC ACETATE"]:
        print(i)
        if type(i) != list:
            dic_drug_id[i] = find_cid(i)
            print(i + ":" + str(dic_drug_id[i])) 
            

In [120]:
len(dic_drug_id)

1559

In [123]:
dic_drug_id
drug_name_list = []
pubchem_id_list = []

for keys in dic_drug_id:
    drug_name_list.append(keys)
    pubchem_id_list.append(dic_drug_id[keys])
    

In [124]:
result = pd.DataFrame({'Drug_name':drug_name_list, 'Pubchem_compound':pubchem_id_list})

In [125]:
result.to_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/Drug_Target/FDA_approved_drugs_withPubchem_id.csv")